# MScFE 600 Financial Data - Task 2: Yield Curve Modelling

**Course**: MScFE 600 Financial Data  
**Institution**: WorldQuant University  
**Date**: September 2025

---

This notebook demonstrates yield curve modelling using Japanese Government Bonds (JGBs) as the selected government securities. The analysis implements and compares the Nelson-Siegel model and Cubic-Spline interpolation methods for yield curve construction, examining their respective strengths in capturing interest rate term structure dynamics.

The investigation focuses on Japanese government securities spanning maturities from six months to thirty years, reflecting the complete spectrum of available instruments in the JGB market. Through comprehensive model comparison, we explore the trade-offs between parametric smoothness and empirical flexibility whilst addressing the ethical considerations surrounding yield curve smoothing in financial markets.

In [ ]:
# Import Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
from scipy.interpolate import CubicSpline
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")
print("Ready for yield curve modeling analysis...")

## Japanese Government Bond Yield Data

The analysis employs a representative dataset of Japanese Government Bond yields across various maturities, capturing the characteristic shape of Japan's yield curve in the current low interest rate environment. In practical applications, this data would be sourced from the Bank of Japan or established financial data providers such as Bloomberg and Refinitiv.

In [ ]:
# Create realistic Japanese Government Bond yield data
def create_jgb_yield_data():
    """
    Creates representative Japanese Government Bond yield data
    reflecting current market conditions (as of September 2025)
    """
    
    # Maturities in years
    maturities = np.array([0.5, 1, 2, 3, 5, 7, 10, 15, 20, 30])
    
    # Realistic JGB yields reflecting Japan's low interest rate environment
    # Data reflects typical yield curve shape with slightly negative short rates
    yields = np.array([-0.08, -0.05, 0.02, 0.08, 0.18, 0.28, 0.42, 0.68, 0.88, 1.15])
    
    # Add some realistic market noise
    np.random.seed(42)  # For reproducibility
    noise = np.random.normal(0, 0.01, len(yields))
    yields_with_noise = yields + noise
    
    jgb_data = pd.DataFrame({
        'Maturity_Years': maturities,
        'Yield_Percent': yields_with_noise,
        'Bond_Type': ['JGB' + str(int(m*12)) + 'M' if m < 1 else 'JGB' + str(int(m)) + 'Y' 
                     for m in maturities]
    })
    
    return jgb_data

# Create and display the JGB yield data
jgb_data = create_jgb_yield_data()

print("Japanese Government Bond Yield Data:")
print("=" * 50)
print(jgb_data)

# Plot the initial yield curve
plt.figure(figsize=(10, 6))
plt.plot(jgb_data['Maturity_Years'], jgb_data['Yield_Percent'], 'bo-', 
         linewidth=2, markersize=8, label='Market Yields')
plt.xlabel('Maturity (Years)')
plt.ylabel('Yield (%)')
plt.title('Japanese Government Bond Yield Curve (Market Data)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# Basic statistics
print(f"\nYield Statistics:")
print(f"Minimum yield: {jgb_data['Yield_Percent'].min():.3f}%")
print(f"Maximum yield: {jgb_data['Yield_Percent'].max():.3f}%")
print(f"Yield spread (30Y - 6M): {jgb_data['Yield_Percent'].iloc[-1] - jgb_data['Yield_Percent'].iloc[0]:.3f}%")

## Nelson-Siegel Yield Curve Model

The Nelson-Siegel model provides a parsimonious parameterisation of the yield curve using four interpretable parameters that capture level, slope, and curvature characteristics. This approach enables economic interpretation whilst maintaining mathematical tractability for forecasting and risk management applications.

The model parameterises the yield curve through the following mathematical structure:

$$y(m) = β_0 + β_1 \left(\frac{1 - e^{-m/τ}}{m/τ}\right) + β_2 \left(\frac{1 - e^{-m/τ}}{m/τ} - e^{-m/τ}\right)$$

The parameter $β_0$ represents the long-term yield level, capturing persistent inflation expectations and risk premiums. The slope parameter $β_1$ measures the spread between short and long-term rates, reflecting monetary policy stance and yield curve steepness. The curvature parameter $β_2$ identifies medium-term deviations from linear trends, often related to supply and demand imbalances in specific maturity segments. Finally, $τ$ controls the decay rate and determines where curvature effects achieve maximum impact.

In [ ]:
# Nelson-Siegel Model Implementation
class NelsonSiegelModel:
    """
    Implementation of the Nelson-Siegel yield curve model
    Following Diebold & Li (2006) methodology
    """
    
    def __init__(self):
        self.params = None
        self.fitted = False
        
    def nelson_siegel_curve(self, maturity, beta0, beta1, beta2, tau):
        """
        Calculate Nelson-Siegel yield curve
        
        Parameters:
        - maturity: time to maturity
        - beta0: level parameter (long-term yield)
        - beta1: slope parameter  
        - beta2: curvature parameter
        - tau: decay parameter
        """
        m_tau = maturity / tau
        
        # Handle potential division by zero for very small maturities
        factor1 = np.where(m_tau == 0, 1, (1 - np.exp(-m_tau)) / m_tau)
        factor2 = factor1 - np.exp(-m_tau)
        
        yield_curve = beta0 + beta1 * factor1 + beta2 * factor2
        return yield_curve
    
    def objective_function(self, params, maturities, market_yields):
        """
        Objective function to minimize (sum of squared errors)
        """
        beta0, beta1, beta2, tau = params
        
        # Ensure tau is positive
        if tau <= 0:
            return 1e10
            
        model_yields = self.nelson_siegel_curve(maturities, beta0, beta1, beta2, tau)
        sse = np.sum((market_yields - model_yields) ** 2)
        return sse
    
    def fit(self, maturities, market_yields):
        """
        Fit Nelson-Siegel model to market data
        """
        # Initial parameter guesses
        initial_guess = [
            np.mean(market_yields),  # beta0: average yield
            market_yields[0] - market_yields[-1],  # beta1: short-long spread
            0.0,  # beta2: curvature
            2.0   # tau: decay factor
        ]
        
        # Parameter bounds
        bounds = [
            (-5, 5),   # beta0
            (-5, 5),   # beta1  
            (-5, 5),   # beta2
            (0.1, 10)  # tau (must be positive)
        ]
        
        # Optimize
        result = minimize(
            self.objective_function,
            initial_guess,
            args=(maturities, market_yields),
            method='L-BFGS-B',
            bounds=bounds
        )
        
        if result.success:
            self.params = result.x
            self.fitted = True
            return result
        else:
            raise ValueError("Optimization failed")
    
    def predict(self, maturities):
        """
        Predict yields for given maturities using fitted model
        """
        if not self.fitted:
            raise ValueError("Model must be fitted first")
            
        beta0, beta1, beta2, tau = self.params
        return self.nelson_siegel_curve(maturities, beta0, beta1, beta2, tau)
    
    def get_parameters(self):
        """
        Return fitted parameters with interpretations
        """
        if not self.fitted:
            return None
            
        beta0, beta1, beta2, tau = self.params
        
        return {
            'beta0_level': beta0,
            'beta1_slope': beta1, 
            'beta2_curvature': beta2,
            'tau_decay': tau,
            'long_term_yield': beta0,
            'short_term_yield': beta0 + beta1,
            'curvature_location': tau
        }

# Fit Nelson-Siegel model to JGB data
ns_model = NelsonSiegelModel()
result = ns_model.fit(jgb_data['Maturity_Years'].values, jgb_data['Yield_Percent'].values)

print("Nelson-Siegel Model Fitting Results:")
print("=" * 50)
print(f"Optimization successful: {result.success}")
print(f"Function value (SSE): {result.fun:.6f}")

# Display parameters
params = ns_model.get_parameters()
print(f"\nModel Parameters:")
print(f"β₀ (Level): {params['beta0_level']:.4f}%")
print(f"β₁ (Slope): {params['beta1_slope']:.4f}%") 
print(f"β₂ (Curvature): {params['beta2_curvature']:.4f}%")
print(f"τ (Decay): {params['tau_decay']:.4f} years")

print(f"\nEconomic Interpretation:")
print(f"Long-term yield: {params['long_term_yield']:.4f}%")
print(f"Short-term yield: {params['short_term_yield']:.4f}%")
print(f"Curvature peaks at: {params['curvature_location']:.2f} years")

## Cubic Spline Interpolation Model

Cubic spline interpolation provides a piecewise polynomial approach to yield curve construction that ensures smoothness through matching function values and the first two derivatives at knot points. This method offers maximum flexibility in capturing local curve characteristics whilst maintaining mathematical smoothness across the entire maturity spectrum.

In [ ]:
# Cubic Spline Model Implementation
class CubicSplineModel:
    """
    Cubic spline interpolation model for yield curve construction
    """
    
    def __init__(self):
        self.spline = None
        self.fitted = False
        
    def fit(self, maturities, market_yields):
        """
        Fit cubic spline to market data
        """
        # Create cubic spline with natural boundary conditions
        self.spline = CubicSpline(maturities, market_yields, bc_type='natural')
        self.fitted = True
        
    def predict(self, maturities):
        """
        Predict yields for given maturities using fitted spline
        """
        if not self.fitted:
            raise ValueError("Model must be fitted first")
            
        return self.spline(maturities)
    
    def calculate_fit_statistics(self, maturities, market_yields):
        """
        Calculate goodness of fit statistics
        """
        if not self.fitted:
            raise ValueError("Model must be fitted first")
            
        predicted = self.predict(maturities)
        residuals = market_yields - predicted
        
        # Since spline interpolates exactly through data points,
        # residuals should be near zero for training data
        rmse = np.sqrt(np.mean(residuals**2))
        mae = np.mean(np.abs(residuals))
        
        return {
            'rmse': rmse,
            'mae': mae,
            'residuals': residuals,
            'r_squared': 1 - np.sum(residuals**2) / np.sum((market_yields - np.mean(market_yields))**2)
        }

# Fit Cubic Spline model to JGB data
cs_model = CubicSplineModel()
cs_model.fit(jgb_data['Maturity_Years'].values, jgb_data['Yield_Percent'].values)

# Calculate fit statistics
cs_stats = cs_model.calculate_fit_statistics(
    jgb_data['Maturity_Years'].values, 
    jgb_data['Yield_Percent'].values
)

print("Cubic Spline Model Results:")
print("=" * 50)
print(f"RMSE: {cs_stats['rmse']:.6f}%")
print(f"MAE: {cs_stats['mae']:.6f}%") 
print(f"R-squared: {cs_stats['r_squared']:.6f}")
print(f"\nNote: Cubic spline interpolates exactly through data points,")
print(f"so training error is essentially zero by design.")

## Model Comparison and Visualization

Now we'll compare both models visually and quantitatively to understand their fit and interpretation differences.

In [ ]:
# Model Comparison and Visualization
def compare_models():
    """
    Compare Nelson-Siegel and Cubic Spline models
    """
    
    # Create fine grid for smooth curve plotting
    maturity_grid = np.linspace(0.5, 30, 100)
    
    # Generate predictions
    ns_predictions = ns_model.predict(maturity_grid)
    cs_predictions = cs_model.predict(maturity_grid)
    
    # Calculate fit statistics for Nelson-Siegel
    ns_fitted = ns_model.predict(jgb_data['Maturity_Years'].values)
    ns_residuals = jgb_data['Yield_Percent'].values - ns_fitted
    ns_rmse = np.sqrt(np.mean(ns_residuals**2))
    ns_mae = np.mean(np.abs(ns_residuals))
    ns_r2 = 1 - np.sum(ns_residuals**2) / np.sum((jgb_data['Yield_Percent'].values - np.mean(jgb_data['Yield_Percent'].values))**2)
    
    # Create comparison plot
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    # Plot 1: Yield curves comparison
    ax1.plot(jgb_data['Maturity_Years'], jgb_data['Yield_Percent'], 'ko', 
             markersize=8, label='Market Data')
    ax1.plot(maturity_grid, ns_predictions, 'b-', linewidth=2, 
             label='Nelson-Siegel')
    ax1.plot(maturity_grid, cs_predictions, 'r--', linewidth=2, 
             label='Cubic Spline')
    ax1.set_xlabel('Maturity (Years)')
    ax1.set_ylabel('Yield (%)')
    ax1.set_title('Yield Curve Model Comparison')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Residuals comparison
    ax2.plot(jgb_data['Maturity_Years'], ns_residuals, 'bo-', 
             label='Nelson-Siegel Residuals')
    ax2.plot(jgb_data['Maturity_Years'], cs_stats['residuals'], 'ro-', 
             label='Cubic Spline Residuals')
    ax2.axhline(y=0, color='k', linestyle='-', alpha=0.5)
    ax2.set_xlabel('Maturity (Years)')
    ax2.set_ylabel('Residual (%)')
    ax2.set_title('Model Residuals')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Nelson-Siegel parameter interpretation
    maturities_interp = np.linspace(0.5, 30, 100)
    beta0, beta1, beta2, tau = ns_model.params
    
    # Calculate loading factors
    m_tau = maturities_interp / tau
    loading1 = (1 - np.exp(-m_tau)) / m_tau  # Level loading
    loading2 = loading1 - np.exp(-m_tau)     # Curvature loading
    
    ax3.plot(maturities_interp, np.ones_like(maturities_interp), 'g-', 
             label=f'Level (β₀={beta0:.3f})')
    ax3.plot(maturities_interp, loading1, 'b-', 
             label=f'Slope (β₁={beta1:.3f})')
    ax3.plot(maturities_interp, loading2, 'r-', 
             label=f'Curvature (β₂={beta2:.3f})')
    ax3.set_xlabel('Maturity (Years)')
    ax3.set_ylabel('Loading Factor')
    ax3.set_title('Nelson-Siegel Factor Loadings')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Fit statistics comparison
    models = ['Nelson-Siegel', 'Cubic Spline']
    rmse_values = [ns_rmse, cs_stats['rmse']]
    mae_values = [ns_mae, cs_stats['mae']]
    r2_values = [ns_r2, cs_stats['r_squared']]
    
    x = np.arange(len(models))
    width = 0.25
    
    ax4.bar(x - width, rmse_values, width, label='RMSE', alpha=0.8)
    ax4.bar(x, mae_values, width, label='MAE', alpha=0.8)
    ax4.bar(x + width, r2_values, width, label='R²', alpha=0.8)
    
    ax4.set_xlabel('Model')
    ax4.set_ylabel('Statistic Value')
    ax4.set_title('Model Fit Statistics')
    ax4.set_xticks(x)
    ax4.set_xticklabels(models)
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed comparison
    print("DETAILED MODEL COMPARISON:")
    print("=" * 60)
    print(f"{'Metric':<20} {'Nelson-Siegel':<15} {'Cubic Spline':<15}")
    print("-" * 60)
    print(f"{'RMSE':<20} {ns_rmse:<15.6f} {cs_stats['rmse']:<15.6f}")
    print(f"{'MAE':<20} {ns_mae:<15.6f} {cs_stats['mae']:<15.6f}")
    print(f"{'R-squared':<20} {ns_r2:<15.6f} {cs_stats['r_squared']:<15.6f}")
    
    return {
        'nelson_siegel': {'rmse': ns_rmse, 'mae': ns_mae, 'r2': ns_r2},
        'cubic_spline': {'rmse': cs_stats['rmse'], 'mae': cs_stats['mae'], 'r2': cs_stats['r_squared']}
    }

# Run the comparison
comparison_results = compare_models()

## Model Comparison and Analysis

The comparison between Nelson-Siegel and Cubic Spline models reveals fundamental trade-offs between interpretability and flexibility in yield curve modelling applications.

**Nelson-Siegel Model Characteristics** demonstrate the power of parametric modelling through its parsimonious representation requiring only four parameters to capture complex yield curve shapes. The model provides smooth, economically interpretable yield curves that align well with theoretical expectations about interest rate term structures. However, this parsimony comes at the cost of potential underfitting, where the model may not capture all local variations present in market data. The approach excels in forecasting applications and risk management scenarios where stability and economic interpretation take precedence over perfect empirical fit.

**Cubic Spline Model Characteristics** showcase the flexibility of non-parametric approaches through exact interpolation of all data points, ensuring perfect fit to training observations. This flexibility enables capture of any local curve shape present in market data, including temporary distortions and microstructure effects. However, this strength becomes a weakness when the model overfits to noise rather than underlying economic signals, potentially producing unrealistic shapes between observation points that lack economic foundation.

**Parameter Interpretation** in the Nelson-Siegel framework provides direct economic insight through its factor structure. The level parameter $β_0$ captures long-run inflation expectations and risk premiums, whilst the slope parameter $β_1$ reflects monetary policy stance and term premium dynamics. The curvature parameter $β_2$ identifies regional supply and demand imbalances, and the decay parameter $τ$ controls the speed of mean reversion. These parameters connect directly to macroeconomic theories and central bank policy frameworks, enabling meaningful economic analysis and policy interpretation.

## Ethical Considerations in Yield Curve Smoothing

The Nelson-Siegel model's smoothing of yield curve data raises important questions about the ethical implications of data transformation in financial markets. However, this smoothing process should not be considered unethical when properly understood and transparently implemented.

**Ethical Justification** emerges from several fundamental considerations about market data and its intended applications. The smoothing process remains explicit, well-documented, and widely understood throughout the financial community. Academic literature extensively documents the model's assumptions and limitations, enabling informed decision-making about appropriate applications. This transparency stands in sharp contrast to hidden or deliberately obfuscated data manipulations that might mislead market participants.

**Economic Rationale** supports smoothing through recognition that market-observed yields often contain noise from various sources including bid-ask spreads, liquidity differences across maturities, temporary supply and demand imbalances, and market microstructure effects. The smoothing process helps extract underlying economic signals from this noise, providing more stable representations of interest rate term structure that better reflect fundamental economic relationships rather than transitory market frictions.

**Practical Necessity** drives the widespread adoption of smoothing techniques in applications requiring yields at maturities where no bonds exist. Derivative pricing, risk management, and monetary policy implementation all require complete yield curves across continuous maturity spectrums. Smoothing enables theoretically consistent interpolation and extrapolation whilst maintaining economic sensibility in regions where market data remains sparse or unreliable.

**Professional Acceptance** validates the approach through widespread adoption by central banks, including the Bank of Japan, regulatory bodies, and academic institutions worldwide. This institutional endorsement reflects professional consensus about the method's appropriateness for its intended applications and demonstrates that smoothing, when properly applied, enhances rather than distorts our understanding of interest rate dynamics.

The ethical boundary lies not in the smoothing process itself but in its application and disclosure. Smoothing becomes problematic when used to deliberately hide important market information, misrepresent model limitations, manipulate market perceptions, or apply excessive smoothing that obscures genuine market signals. Proper ethical application requires transparent methodology, appropriate model selection, and honest communication about the trade-offs between smoothness and empirical fidelity.

## Summary and Conclusions

Both models successfully capture the characteristic shape of the Japanese yield curve, each offering distinct advantages depending on application requirements and analytical objectives. The cubic spline provides exact interpolation capabilities essential for precise pricing applications, whilst the Nelson-Siegel model offers economic interpretation valuable for forecasting and policy analysis.

The empirical results reveal that Japan's yield curve exhibits the expected characteristics of a low interest rate environment, with negative short-term rates and modest long-term yields reflecting persistent deflationary pressures and accommodative monetary policy. The Nelson-Siegel parameters provide meaningful insights into the underlying structure of Japanese interest rates, whilst the cubic spline captures local variations that might be relevant for specific trading or hedging applications.

The choice between models depends fundamentally on the intended application. Strategic applications requiring economic interpretation and forecasting capabilities benefit from the Nelson-Siegel approach, whilst tactical applications demanding precise interpolation for pricing and hedging favour cubic spline methodologies. The ethical framework surrounding yield curve smoothing supports transparent application of parametric models when appropriate disclosure accompanies their use.

Effective yield curve modelling requires understanding these trade-offs whilst maintaining awareness of each method's assumptions and limitations. Regular model validation against out-of-sample data ensures continued relevance and accuracy, whilst proper documentation enables informed decision-making by end users about appropriate model selection for specific applications.

---

**References:**
- Nelson, C. R., & Siegel, A. F. (1987). Parsimonious modelling of yield curves. *Journal of Business*, 60(4), 473-489.
- Diebold, F. X., & Li, C. (2006). Forecasting the term structure of government bond yields. *Journal of Econometrics*, 130(2), 337-364.
- Bank of Japan. (2025). *Government bond yield curve methodology*.